In [15]:
import numpy as np
import h5py
import pandas as pd

In [16]:
mergers = np.load('Data/ill-1_blackhole_mergers_fixed.npz')
evol = h5py.File("Data/mbhb-evolution_no-ecc_lc-full-0.6.hdf5","r")

In [58]:
Ms = evol['evolution']['masses']
t = evol['evolution']['times']
r = evol['evolution']['sep']

In [17]:
indexes = evol['evolution']['val_inds'][:]
binary_ids = mergers['ids'][indexes]

In [22]:
N_binary = np.shape(binary_ids)[0]

In [61]:
M1_list = []
M2_list = []
t_list = []

M1_list = []
M2_list = []
t_list = []

for i in range(N_binary):
    M1_list.append(Ms[i][0])
    M2_list.append(Ms[i][1])
    t_list.append(t[i][np.argwhere(r[i]==0)[0][0]]) #t merger is when the seperation goes to 0


In [84]:
from astropy.cosmology import WMAP9 as cosmo
import astropy.units as u
Hinverse = cosmo.H(0).to(1/u.s)
hubble_time = (1/cosmo.H(0)).to(u.yr).value #Hubble time = 1/H0
print(hubble_time)

14105485021.361647


In [98]:
yrsec = (1*u.yr).to(u.s).value # 1 year in secs
t_list = t_list/yrsec

In [103]:
stalled_count = 0
merger=[]

for time in t_list:
    if time >= hubble_time:
        #these black holes aren't merging
        stalled_count = stalled_count + 1
        merger.append("No")
    else:
        merger.append("Yes")

print("Total stalled MBHs %d"%(stalled_count))

Total stalled MBHs 6809


In [ ]:
Triple_df = pd.read_csv("Data/triple_data_ill.csv")

In [28]:
bh1id1 = Triple_df["BH1_ID1"].to_numpy()
bh1id2 = Triple_df["BH1_ID2"].to_numpy()
bh2id1 = Triple_df["BH2_ID1"].to_numpy()
bh2id2 = Triple_df["BH2_ID2"].to_numpy()

In [157]:
print(bh1id1[0],bh1id2[0],bh2id1[0],bh2id2[0])

9.22337203709081e+18 9.223372037238491e+18 9.223372036720252e+18 9.223372037238491e+18


In [41]:
def searchID(id):

    tf1 = id in bh1id1
    tf2 = id in bh1id2
    tf3 = id in bh2id1
    tf4 = id in bh2id2

    tf = tf1 + tf2 + tf3 + tf4

    return tf 


In [52]:
type = []

for i in range(N_binary):

    search1 = searchID(binary_ids[i][0])
    search2 = searchID(binary_ids[i][1])

    search = search1 + search2
    if(search == 0):
        type.append("iso")
    else:
        type.append("trip")


In [114]:
df = pd.DataFrame([M1_list,M2_list,t_list,merger,type])
df = df.transpose()
df.columns = ['M1','M2','t_merger','Merger','Type']

In [115]:
df.head()

,M1,M2,t_merger,Merger,Type
0,4496150597824015233342643531839143149568.0,3239130977150481045247849004617980444672.0,858774126.450153,Yes,iso
1,3210830029843102916184613152605259956224.0,2142212244418209772291345238083162341376.0,893400572.050411,Yes,iso
2,13033744257841817886100508794345280765952.0,2337819241002291541715754422103479156736.0,4120229485.36754,Yes,iso
3,13997642889337732808502100799021612269568.0,12454619883001010817570679262288589881344.0,89337138643.547943,No,iso
4,23446120332452297114271153103284404748288.0,6574773268621633779839470064839258275840.0,14217143975.611784,No,iso


In [135]:
df_iso = df[(df['Type']=='trip')]['Merger'] == "No"
print(df_iso.sum())

919


In [145]:
(df[(df["Type"]=="iso")]['Merger'] == "Yes").sum()

1998

In [146]:
(df[(df["Type"]=="trip")]['Merger'] == "Yes").sum()

427

In [148]:
(df[(df['Merger'] == "No")]['Type']=="trip").sum()

919

In [149]:
(df[(df['Merger'] == "No")]['Type']=="iso").sum()

5890